<a href="https://colab.research.google.com/github/devloperAnu/Sample_to_target/blob/main/sample_to_target.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell 1: Install Dependencies
!pip install so-vits-svc-fork
!pip install pydub
!apt-get install -y ffmpeg
print("Dependencies installed successfully!")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
Dependencies installed successfully!


# New Section

In [1]:
# Cell 2: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
print("Google Drive mounted successfully!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted successfully!


In [2]:
# Cell 3: Create Directory Structure
import os

# Define paths
base_path = '/content/drive/MyDrive/so-vits-svc-fork'
voice_sample_dir = os.path.join(base_path, 'Audio_file.mp3')
input_audio_dir = os.path.join(base_path, 'output')
output_dir = os.path.join(base_path, 'target.mp3')

# Create directories if they don't exist
os.makedirs(voice_sample_dir, exist_ok=True)
os.makedirs(input_audio_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

print(f"Directories created:\nVoice Samples: {voice_sample_dir}\nInput Audio: {input_audio_dir}\nOutput: {output_dir}")

Directories created:
Voice Samples: /content/drive/MyDrive/so-vits-svc-fork/Audio_file.mp3
Input Audio: /content/drive/MyDrive/so-vits-svc-fork/output
Output: /content/drive/MyDrive/so-vits-svc-fork/target.mp3


In [3]:
# Install Python libraries for audio processing
!pip install librosa pydub soundfile numpy

# Install ffmpeg for pydub to process MP3 files
!apt-get install -y ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [4]:
# Import libraries
import librosa
import soundfile as sf
from pydub import AudioSegment
import numpy as np
import os
from google.colab import files

In [6]:
from google.colab import files

# Upload sample.mp3
print("Please upload sample.mp3")
sample_upload = files.upload()
if not sample_upload:
    raise ValueError("Please upload sample.mp3")
sample_file = list(sample_upload.keys())[0]

# Upload target.mp3
print("Please upload target.mp3")
target_upload = files.upload()
if not target_upload:
    raise ValueError("Please upload target.mp3")
target_file = list(target_upload.keys())[0]

Please upload sample.mp3


Saving sample.mp3 to sample.mp3
Please upload target.mp3


Saving target.mp3 to target.mp3


In [8]:
import os
from pydub import AudioSegment

# Define file paths (these could be dynamic, e.g., from user input or uploaded files)
sample_file = "sample.mp3"
target_file = "target.mp3"
sample_wav = "sample.wav"
target_wav = "target.wav"

# Function to convert MP3 to WAV
def mp3_to_wav(mp3_path: str, wav_path: str) -> None:
    """
    Convert an MP3 file to WAV format.

    Args:
        mp3_path (str): Path to the input MP3 file.
        wav_path (str): Path to save the output WAV file.

    Raises:
        FileNotFoundError: If the input MP3 file does not exist.
        ValueError: If the input file is not an MP3 file.
        RuntimeError: If conversion fails due to invalid file or missing dependencies.
    """
    # Validate input file
    if not os.path.exists(mp3_path):
        raise FileNotFoundError(f"Input file not found: {mp3_path}")
    if not mp3_path.lower().endswith('.mp3'):
        raise ValueError(f"Input file must be an MP3: {mp3_path}")

    # Check if output file already exists
    if os.path.exists(wav_path):
        print(f"Warning: Overwriting existing file: {wav_path}")

    try:
        # Load and convert the MP3 file
        audio = AudioSegment.from_mp3(mp3_path)
        audio.export(wav_path, format="wav")
        print(f"Successfully converted {mp3_path} to {wav_path}")
    except Exception as e:
        raise RuntimeError(f"Failed to convert {mp3_path} to WAV: {str(e)}")

# Convert both files to WAV with error handling
try:
    mp3_to_wav(sample_file, sample_wav)
    mp3_to_wav(target_file, target_wav)
except Exception as e:
    print(f"Error: {str(e)}")

Error: Failed to convert sample.mp3 to WAV: Decoding failed. ffmpeg returned error code: 1

Output from ffmpeg/avlib:

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libs

In [10]:
import os
import numpy as np
import librosa

# Define file path (ensure this is defined or passed dynamically)
sample_wav = "sample.wav"

# Function to analyze audio pitch
def analyze_audio(file_path: str) -> dict:
    """
    Analyze the pitch (fundamental frequency) of an audio file.

    Args:
        file_path (str): Path to the input audio file (e.g., WAV).

    Returns:
        dict: Dictionary containing the estimated pitch (in Hz) and sampling rate.

    Raises:
        FileNotFoundError: If the input file does not exist.
        ValueError: If the input file is not a WAV file or is invalid.
        RuntimeError: If audio loading or pitch analysis fails.
    """
    # Validate input file
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Audio file not found: {file_path}")
    if not file_path.lower().endswith('.wav'):
        raise ValueError(f"Input file must be a WAV file: {file_path}")

    try:
        # Load audio file
        y, sr = librosa.load(file_path, sr=None)  # Use native sampling rate
    except Exception as e:
        raise RuntimeError(f"Failed to load audio file {file_path}: {str(e)}")

    try:
        # Extract pitch (fundamental frequency) using piptrack
        pitches, magnitudes = librosa.piptrack(y=y, sr=sr)

        # Get the most prominent pitch per frame (max magnitude)
        if np.any(magnitudes > 0):
            # Find the index of the maximum magnitude for each frame
            max_magnitude_indices = np.argmax(magnitudes, axis=0)
            # Extract corresponding pitches for frames with non-zero magnitude
            valid_pitches = pitches[max_magnitude_indices, np.arange(pitches.shape[1])]
            valid_pitches = valid_pitches[magnitudes[max_magnitude_indices, np.arange(pitches.shape[1])] > 0]
            # Compute mean pitch (excluding zeros)
            pitch = np.mean(valid_pitches) if valid_pitches.size > 0 else 0
        else:
            print(f"Warning: No valid pitch detected in {file_path}")
            pitch = 0

        return {"pitch": pitch, "sr": sr}
    except Exception as e:
        raise RuntimeError(f"Failed to analyze pitch for {file_path}: {str(e)}")

# Analyze sample audio with error handling
try:
    sample_features = analyze_audio(sample_wav)
    print(f"Sample audio pitch: {sample_features['pitch']:.2f} Hz (Sampling rate: {sample_features['sr']} Hz)")
except Exception as e:
    print(f"Error: {str(e)}")

Error: Audio file not found: sample.wav


In [12]:
import os
import numpy as np
import librosa
import soundfile as sf

# Define file paths and variables (ensure these are defined or passed dynamically)
target_wav = "target.wav"
converted_wav = "converted.wav"
sample_features = {"pitch": 440.0, "sr": 44100}  # Example placeholder; replace with actual sample_features

# Function to analyze pitch of an audio file
def analyze_pitch(file_path: str, sr: int = None) -> float:
    """
    Analyze the pitch (fundamental frequency) of an audio file.

    Args:
        file_path (str): Path to the input audio file (e.g., WAV).
        sr (int, optional): Sampling rate. If None, use native sampling rate.

    Returns:
        float: Estimated pitch in Hz, or 0 if no valid pitch is detected.

    Raises:
        FileNotFoundError: If the input file does not exist.
        ValueError: If the input file is not a WAV file.
        RuntimeError: If audio loading or pitch analysis fails.
    """
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Audio file not found: {file_path}")
    if not file_path.lower().endswith('.wav'):
        raise ValueError(f"Input file must be a WAV file: {file_path}")

    try:
        y, sr = librosa.load(file_path, sr=sr)
        pitches, magnitudes = librosa.piptrack(y=y, sr=sr)
        if np.any(magnitudes > 0):
            max_magnitude_indices = np.argmax(magnitudes, axis=0)
            valid_pitches = pitches[max_magnitude_indices, np.arange(pitches.shape[1])]
            valid_pitches = valid_pitches[magnitudes[max_magnitude_indices, np.arange(pitches.shape[1])] > 0]
            pitch = np.mean(valid_pitches) if valid_pitches.size > 0 else 0
        else:
            pitch = 0
        if pitch == 0:
            print(f"Warning: No valid pitch detected in {file_path}")
        return pitch
    except Exception as e:
        raise RuntimeError(f"Failed to analyze pitch for {file_path}: {str(e)}")

# Process target audio and apply pitch shift
try:
    # Validate input file
    if not os.path.exists(target_wav):
        raise FileNotFoundError(f"Target audio file not found: {target_wav}")

    # Load target audio
    y_target, sr_target = librosa.load(target_wav, sr=None)

    # Analyze target audio pitch
    target_pitch = analyze_pitch(target_wav, sr=None)
    print(f"Target audio pitch: {target_pitch:.2f} Hz")

    # Calculate pitch shift (in semitones)
    if target_pitch > 0 and sample_features['pitch'] > 0:
        n_steps = 12 * np.log2(sample_features['pitch'] / target_pitch)
        print(f"Pitch shift required: {n_steps:.2f} semitones")
    else:
        print("Warning: Cannot compute pitch shift (invalid pitch detected)")
        n_steps = 0

    # Validate pitch shift parameters
    if abs(n_steps) > 24:  # Arbitrary limit to avoid extreme shifts
        print(f"Warning: Pitch shift ({n_steps:.2f} semitones) is too large; limiting to ±24 semitones")
        n_steps = max(min(n_steps, 24), -24)

    # Apply pitch shift to target audio
    y_shifted = librosa.effects.pitch_shift(y=y_target, sr=sr_target, n_steps=n_steps)

    # Save the modified audio
    try:
        sf.write(converted_wav, y_shifted, sr_target)
        print(f"Modified audio saved to {converted_wav}")
    except Exception as e:
        raise RuntimeError(f"Failed to save audio to {converted_wav}: {str(e)}")

except Exception as e:
    print(f"Error: {str(e)}")

Error: Target audio file not found: target.wav


In [14]:
import os
from pydub import AudioSegment

# Define file paths (ensure these are defined or passed dynamically)
converted_wav = "converted.wav"
output_file = "output.mp3"
sample_wav = "sample.wav"
target_wav = "target.wav"

# Function to convert WAV to MP3
def wav_to_mp3(wav_path: str, mp3_path: str) -> None:
    """
    Convert a WAV file to MP3 format.

    Args:
        wav_path (str): Path to the input WAV file.
        mp3_path (str): Path to save the output MP3 file.

    Raises:
        FileNotFoundError: If the input WAV file does not exist.
        ValueError: If the input file is not a WAV file or output is not an MP3 file.
        RuntimeError: If conversion fails due to invalid file or missing dependencies.
    """
    # Validate input file
    if not os.path.exists(wav_path):
        raise FileNotFoundError(f"Input WAV file not found: {wav_path}")
    if not wav_path.lower().endswith('.wav'):
        raise ValueError(f"Input file must be a WAV file: {wav_path}")

    # Validate output file
    if not mp3_path.lower().endswith('.mp3'):
        raise ValueError(f"Output file must be an MP3 file: {mp3_path}")
    if os.path.exists(mp3_path):
        print(f"Warning: Overwriting existing file: {mp3_path}")

    try:
        # Load WAV file and export as MP3
        audio = AudioSegment.from_wav(wav_path)
        audio.export(mp3_path, format="mp3")
        print(f"Successfully converted {wav_path} to {mp3_path}")
    except Exception as e:
        raise RuntimeError(f"Failed to convert {wav_path} to MP3: {str(e)}")

# Convert modified audio to MP3 and clean up temporary files
try:
    # Convert WAV to MP3
    wav_to_mp3(converted_wav, output_file)

    # Clean up temporary WAV files
    for temp_file in [sample_wav, target_wav, converted_wav]:
        if os.path.exists(temp_file):
            try:
                os.remove(temp_file)
                print(f"Deleted temporary file: {temp_file}")
            except Exception as e:
                print(f"Warning: Failed to delete {temp_file}: {str(e)}")
        else:
            print(f"Warning: Temporary file not found: {temp_file}")

except Exception as e:
    print(f"Error: {str(e)}")

Error: Input WAV file not found: converted.wav


In [16]:
import os
from google.colab import files

# Define file path (ensure this is defined or passed dynamically)
output_file = "output.mp3"

# Download the output MP3
try:
    # Validate file existence
    if not os.path.exists(output_file):
        raise FileNotFoundError(f"Output file not found: {output_file}")

    print(f"Downloading {output_file}...")
    files.download(output_file)
    print(f"Successfully initiated download of {output_file}")

except FileNotFoundError as e:
    print(f"Error: {str(e)}")
except Exception as e:
    print(f"Error during download: {str(e)}")

Error: Output file not found: output.mp3
